In [19]:
import collections
import math

In [39]:
reference_corpus = [[['this', 'is', 'a', 'test'], ['this', 'is' ,'test']],[['The','cat','is','on','the','mat'],['There','is','a','cat','on', 'the','mat']],[['The','cat','is','on','the','mat'],['There','is','a','cat','on', 'the','mat']],[['The','cat','is','on','the','mat'],['There','is','a','cat','on', 'the','mat']]]

translation_corpus = [['this', 'is', 'a', 'test'],['the','the','the','the','the','the','the'],['the','cat','the','cat','on','the','mat'],['the','cat','on','the','mat']]

In [20]:
reference_corpus = [[['this', 'is', 'a', 'test'], ['this', 'is' ,'test']]]
translation_corpus = [['this', 'is', 'a', 'test']]

In [40]:
max_order=4

In [41]:
matches_by_order = [0] * max_order
possible_matches_by_order = [0] * max_order

In [42]:
matches_by_order

[0, 0, 0, 0]

In [43]:
possible_matches_by_order

[0, 0, 0, 0]

In [44]:
reference_length = 0
translation_length = 0

In [45]:
def _get_ngrams(segment, max_order):
    ngram_counts = collections.Counter()
    for order in range(1, max_order + 1):
        for i in range(0, len(segment) - order + 1):
            ngram = tuple(segment[i:i+order])
            ngram_counts[ngram] += 1
    return ngram_counts

In [46]:
for (references, translation) in zip(reference_corpus,
                                       translation_corpus):
    reference_length += min(len(r) for r in references)
    translation_length += len(translation)

    merged_ref_ngram_counts = collections.Counter()
    for reference in references:
        merged_ref_ngram_counts |= _get_ngrams(reference, max_order)
    translation_ngram_counts = _get_ngrams(translation, max_order)
    overlap = translation_ngram_counts & merged_ref_ngram_counts
    for ngram in overlap:
        matches_by_order[len(ngram)-1] += overlap[ngram]
    for order in range(1, max_order+1):
        possible_matches = len(translation) - order + 1
        if possible_matches > 0:
            possible_matches_by_order[order-1] += possible_matches

In [47]:
overlap

Counter({('cat',): 1,
         ('cat', 'on'): 1,
         ('cat', 'on', 'the'): 1,
         ('cat', 'on', 'the', 'mat'): 1,
         ('mat',): 1,
         ('on',): 1,
         ('on', 'the'): 1,
         ('on', 'the', 'mat'): 1,
         ('the',): 1,
         ('the', 'mat'): 1})

In [48]:
matches_by_order

[13, 9, 6, 3]

In [49]:
possible_matches_by_order

[23, 19, 15, 11]

In [50]:
precisions = [0] * max_order
for i in range(0, max_order):
    if possible_matches_by_order[i] > 0:
        precisions[i] = (float(matches_by_order[i]) / possible_matches_by_order[i])
    else:
        precisions[i] = 0.0


In [51]:
precisions

[0.5652173913043478, 0.47368421052631576, 0.4, 0.2727272727272727]

In [52]:
if min(precisions) > 0:
    p_log_sum = sum((1. / max_order) * math.log(p) for p in precisions)
    geo_mean = math.exp(p_log_sum)
else:
    geo_mean = 0

In [53]:
geo_mean

0.4134026406666258

In [54]:
 ratio = float(translation_length) / reference_length

In [55]:
ratio

1.0952380952380953

In [56]:
if ratio > 1.0:
    bp = 1.
else:
    bp = math.exp(1 - 1. / ratio)

bleu = geo_mean * bp

In [57]:
bleu

0.4134026406666258